<a href="https://colab.research.google.com/github/ksarita957/poetic-style-transfer/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import string
import re
import json
import operator
#Get poems from a specific author with line breaks
def get_author_poems_lb(author):
    ap = []
    url = "https://www.poetryfoundation.org/poets/" + author
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    exclude = set(string.punctuation)
    request=urllib.request.Request(url,None,headers) #The assembled request
    d = 0
    try:
        response = urllib.request.urlopen(request)
    except:
        d = None
    if d == 0:
        data = response.read()
        soup = BeautifulSoup(data,features='html.parser')
        f = soup.find("div", {"class":"o-tabPanel-panel"})
        all_poems = []
        if f:
            for link in f.findAll('a', attrs={'href': re.compile("^https://")}):
                all_poems.append(link.get('href'))
            for a in all_poems:
                request=urllib.request.Request(a,None,headers)
                d = 0
                try:
                    response = urllib.request.urlopen(request)
                except:
                    d = None
                if d == 0:
                    data = response.read()
                    sp = BeautifulSoup(data,features='html.parser')
                    nf = sp.find("div",{"class", "o-poem"})
                    if nf is not None:
                        plist = []
                        s = nf.find_all("div")
                        if s:
                            for rkl in s:
                                j = rkl.get_text()
                                j = ''.join(ch for ch in j if ch not in exclude)
                                plist.append(j.strip().lower().split())
                            if len(plist) > 0:
                                ap.append(plist)
    return ap
#Get poems from specific author no line breaks
def get_author_poems(author):
    plist = []
    url = "https://www.poetryfoundation.org/poets/" + author
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    exclude = set(string.punctuation)
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read()
    soup = BeautifulSoup(data,features='html.parser')
    f = soup.find("div", {"class":"o-tabPanel-panel"})
    all_poems = []
    if f:
        for link in f.findAll('a', attrs={'href': re.compile("^https://")}):
            all_poems.append(link.get('href'))
        for a in all_poems:
            request=urllib.request.Request(a,None,headers)
            response = urllib.request.urlopen(request)
            data = response.read()
            sp = BeautifulSoup(data,features='html.parser')
            nf = sp.find("div",{"class", "o-poem"})
            if nf is not None:
                s = nf.get_text(separator=' ')
                s = ''.join(ch for ch in s if ch not in exclude)
                plist.append(s.lower().split())
    return plist
#Get poems from a url
def get_poem(poem_url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    plist = []
    request=urllib.request.Request(poem_url,None,headers)
    response = urllib.request.urlopen(request)
    data = response.read()
    sp = BeautifulSoup(data,features='html.parser')
    nf = sp.find("div",{"class", "o-poem"})
    exclude = set(string.punctuation)
    if nf is not None:
        s = nf.get_text(separator=' ')
        s = ''.join(ch for ch in s if ch not in exclude)
        plist = s.lower().split()
    return plist


In [ ]:
#Get poems from four authors. Store as list of lists, where each poem is a list of lines which are in turn represented by lists
auts = ['william-shakespeare', 'alfred-tennyson', 'rae-armantrout', 'william-wordsworth']
author_poems = []
author_labels = []
i = 0
while i < len(auts):
  a = get_author_poems_lb(auts[i])
  for j in a:
    author_poems.append(j)
    author_labels.append(i)
  i += 1
r_authors_poem_line = []
i = 0
for p in author_poems:
  pp = []
  for j in p:
    jp = []
    for word in j:
      if '—' not in word:
        jp.append(word)
      else:
        for w in word.split('—'):
          jp.append(w)
    pp.append(jp)
  i += 1
  r_authors_poem_line.append(pp)
print(r_authors_poem_line[0])
print(len(r_authors_poem_line))

In [ ]:
#Creates dictionary of Vocabulary. Each word or token corresponds to a number
from collections import Counter
def create_vocab_dict(poems,mincount=10):
  word_to_idx = {'<pad>':0, '<eos>':1, '<go>':2, '<unk>':3}
  cts = {}
  words = []
  for p in poems:
    for w in p:
      for j in w:
        words.append(j)
  cnt= Counter(words)
  for word in cnt:
    if cnt[word] >= mincount:
      word_to_idx[word]= len(word_to_idx)
  return word_to_idx

#Convert to sentences and turn words into vocab indices
def vnorm(poems,vocab, lab, maxlen=100):
  numpoem = []
  indpoem = []
  lengs = []
  pr = 0
  for p in poems:
    index = lab[pr]
    for line in p:
      a = []
      if len(line) > 0:
        a.append(vocab['<go>'])
        for h in line:
          if h in vocab:
            a.append(vocab[h])
          else:
            a.append(vocab['<unk>'])
        if len(a) > maxlen-1:
          a = a[0:maxlen-1]
          a.append(vocab['<eos>'])
          lengs.append(maxlen)
        else:
          a.append(vocab['<eos>'])
          car = len(a)
          lengs.append(car)
          i = 0
          while i < maxlen-car:
            a.append(vocab['<pad>'])
            i += 1
        numpoem.append(a)
        indpoem.append(index)
    pr += 1
  return numpoem,indpoem,lengs
#Turns sequence of labels into a one hot matrix
def one_hot(labs,dim):
  oh = torch.zeros((len(labs),dim))
  i = 0
  while i < len(labs):
    oh[i][labs[i]] = 1
    i += 1
  return oh
cr = create_vocab_dict(r_authors_poem_line)
padded,inds,lengs = vnorm(r_authors_poem_line,cr,author_labels)
print(len(padded))

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
#VAE with LSTM, style as linear layer 
class LSTM_VAE(nn.Module):
  def __init__(self, input_length, num_classes, hidden_length,layers,vocab_length, gen_length, out_length,drop=0.1,temp=10):
    super(LSTM_VAE,self).__init__()
    #expanding one hot of style label
    self.style_layer = nn.Linear(num_classes,out_length)
    #length of generation: 1 - line length
    self.genlen = gen_length
    #embedding dimension
    self.inp = input_length
    #latent dimension
    self.out = out_length
    #length of vocab
    self.nvocab = vocab_length
    #dropout later
    self.drop = nn.Dropout(p=drop)
    #number of layers RNN
    self.numlayers = layers
    #transforming encoder hidden state to mean
    self.mean_layer = nn.Linear(layers*hidden_length,out_length)
    #transforming decoder hidden state to log variance
    self.var_layer = nn.Linear(layers*hidden_length,out_length)
    #encoder LSTM
    self.enc = nn.LSTM(input_length+out_length,hidden_length,layers,batch_first=True)
    #Decoder LSTM
    self.dec = nn.LSTM(input_length+2*out_length,input_length,batch_first=True)
    #word embeddings
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #transform decoder hidden state to vector of length of vocab
    self.llayer = nn.Linear(input_length,vocab_length)
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #apply embedding to each word
    x = self.embedding(x)
    #expand style to encode style information
    y = self.style_layer(y)
    #concatenate style and poem information
    ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    #pack sequence to ignore pad
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    #get final hidden state from encoder and reshape to two dimensions
    _,(henc,_) = self.enc(ex)
    henc = henc.view((henc.size()[1],henc.size()[2]*self.numlayers))
    #get mean and log variance
    var = self.var_layer(henc)
    mean = self.mean_layer(henc)
    #draw new sample based on reparam trick: z = N(0,I)*var + mean
    draw = torch.normal(torch.zeros(var.size()),torch.ones(var.size())).to(device)
    z = draw*torch.exp(var) + mean
    #caluclate KL-divergence
    kl = 0.5*(torch.mean(-torch.sum(var,axis=1) + torch.sum(torch.exp(var),axis=1) + torch.sum(torch.square(mean),axis=1)-var.size()[1]))
    #get decoder input: ground truth, latent variable, and style concatenated
    inp = torch.zeros((x.size()[0],self.genlen,self.inp + 2*self.out)).to(device)
    i = 0
    while i < self.genlen:
      inp[:,i,:] = torch.cat((z,y,x[:,i,:]),1)
      i += 1
    #dropout in decoder
    inp = self.drop(inp)
    #pack sequence to ignore padding
    inp = pack_padded_sequence(inp,les-1,batch_first=True, enforce_sorted=False)
    #pass through decoder
    gen,_ = self.dec(inp)
    #unpack and apply linear layer, then reshape to match desired shape for CrossEntropyLoss
    gen = pad_packed_sequence(gen,batch_first=True,total_length=self.genlen)[0]
    gen = self.llayer(gen)
    gen = gen.view((gen.size()[0],gen.size()[2],gen.size()[1]))
    #rescale by temperature
    gen = (gen - torch.max(gen,1)[0].unsqueeze(1).repeat(1,len(vocab),1))/self.temp
    return gen,kl



In [ ]:
#VAE with LSTM, style as linear layer training
#Set seeds for reproducability
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#Choose which labels we want to use - only two at a time
a0 = np.where(ind==0)[0]
a1 = np.where(ind==1)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#Divide data into training and validation sets
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#Weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[0] = 0
weights[3] = 0
#Initialize network, loss, and optimizer
net = LSTM_VAE(300,2,300,1,len(vocab.keys()),mlen-1,25,drop=0.1,temp=10)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(),lr=0.0001)
batch_size = 256
trainloss = []
trainkl = []
validloss = []
validkl = []
#coefficients for KL annealing
klcoef = np.concatenate((np.zeros(250),np.linspace(0,1,250),np.ones(9500)))
i = 0
while i < 4792:
  #Reshuffle indices each epoch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #zero optimizer
    optimizer.zero_grad()
    #sample minibatch
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    class_inp = one_hot(labels[train_track[k*batch_size:(k+1)*batch_size]],2).to(device)
    #create labels for reconstruction loss
    oh = inp.long().to(device)
    #pass through network
    gen,kl = net(inp,class_inp,les)
    #record average over nonzero entries in each line and KL divergence
    rec_loss = loss_func(gen,oh[:,1:mlen])
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    klp += kl.item()
    #calculate total loss and complete backward pass
    loss = torch.mean(loss_func(gen,oh[:,1:mlen])) + kl*klcoef[i]
    loss.backward()
    #gradient clip
    torch.nn.utils.clip_grad_norm_(net.parameters(),max_norm=1)
    optimizer.step()
    k += 1
  #Report avg reconstruction and KL divergence loss
  trainloss.append(lp/len(input))
  trainkl.append(klp/len(input))
  #Compute same measures over validation set
  with torch.no_grad():
    k = 0
    while k < len(vind)//batch_size + 1:
      inp = validation[valid_track[k*batch_size:(k+1)*batch_size]]
      les = vlengs[valid_track[k*batch_size:(k+1)*batch_size]]
      oh = inp.long().to(device)
      class_inp = one_hot(vlabels[valid_track[k*batch_size:(k+1)*batch_size]],2).to(device)
      gen,kl = net(inp,class_inp,les)
      rec_loss = loss_func(gen,oh[:,1:mlen])
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      klp += kl.item()
      k += 1
    validloss.append(lp/len(validation))
    validkl.append(klp/len(validation))
  print(i,trainloss[-1],trainkl[-1],validloss[-1],validkl[-1])
  i += 1


In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
#VAE with Bidirectional LSTM, style as linear layer 
class LSTM_VAE_BD(nn.Module):
  def __init__(self, input_length, num_classes, hidden_length,layers,vocab_length, gen_length, out_length,drop=0.1,temp=10):
    super(LSTM_VAE_BD,self).__init__()
    #expanding one hot of style 
    self.style_layer = nn.Linear(num_classes,out_length)
    #length of generation
    self.genlen = gen_length
    #dimension of embedding
    self.inp = input_length
    #latent dimension
    self.out = out_length
    #length of Vocabulary
    self.nvocab = vocab_length
    #dropout layer
    self.drop = nn.Dropout(p=drop)
    #transforming hidden state to mean of latent
    self.mean_layer = nn.Linear(2*hidden_length,out_length)
    #transforming hidden state to logvariance of latent
    self.var_layer = nn.Linear(2*hidden_length,out_length)
    #encoder 
    self.enc = nn.LSTM(input_length+out_length,hidden_length,layers,batch_first=True,bidirectional=True)
    #decoder
    self.dec = nn.LSTM(input_length+2*out_length,input_length,batch_first=True)
    #word embeddings
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #transform decoder hidden state to vector of length of vocab
    self.llayer = nn.Linear(input_length,vocab_length)
    #number of layers
    self.numlayers = layers
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #apply embedding to input
    x = self.embedding(x)
    #expand style information
    y = self.style_layer(y)
    #pack sequence to ignore padding
    ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    #pass through encoder
    _,(henc,_) = self.enc(ex)
    henc = henc.view((henc.size()[1],henc.size()[2]*2*self.numlayers))
    #get mean and log variance of latent dimension
    var = self.var_layer(henc)
    mean = self.mean_layer(henc)
    #draw new sample based on reparam trick: z = N(0,I)*var + mean
    draw = torch.normal(torch.zeros(var.size()),torch.ones(var.size())).to(device)
    z = draw*torch.exp(var) + mean
    #calculate kl divergence
    kl = 0.5*(torch.mean(-torch.sum(var,axis=1) + torch.sum(torch.exp(var),axis=1) + torch.sum(torch.square(mean),axis=1)-var.size()[1]))
    #create input for decoder with teacher forcing - concatenate latent,stlye, and input
    inp = torch.zeros((x.size()[0],self.genlen,self.inp + 2*self.out)).to(device)
    i = 0
    while i < self.genlen:
      inp[:,i,:] = torch.cat((z,y,x[:,i,:]),1)
      i += 1
    #dropout layer
    inp = self.drop(inp)
    #pack sequence for decoder
    inp = pack_padded_sequence(inp,les-1,batch_first=True, enforce_sorted=False)
    #pass through decoder and unpack
    gen,_ = self.dec(inp)
    gen = pad_packed_sequence(gen,batch_first=True,total_length=self.genlen)[0]
    #expand to vector over vocab and reshape for CrossEntropyLoss
    gen = self.llayer(gen)
    gen = gen.view((gen.size()[0],gen.size()[2],gen.size()[1]))
    #rescale by temperature
    gen = (gen - torch.max(gen,1)[0].unsqueeze(1).repeat(1,len(vocab),1))/self.temp
    return gen,kl



In [ ]:
#VAE with LSTM, style as linear layer training
#Set seeds for reproducability
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#Choose which labels we want to use - only two at a time
a0 = np.where(ind==0)[0]
a1 = np.where(ind==1)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#Divide data into training and validation sets
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#Weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[0] = 0
weights[3] = 0
#initialize network,loss, and optimizer
net = LSTM_VAE_BD(300,2,300,1,len(vocab.keys()),mlen-1,25,drop=0.1,temp=10)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(),lr=0.0001)
batch_size = 256
trainloss = []
trainkl = []
validloss = []
validkl = []
#coefficients for annealing
klcoef = np.concatenate((np.zeros(250),np.linspace(0,1,250),np.ones(9500)))
i = 0
while i < 10000:
  #randomize minibatch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #zero out optimizer
    optimizer.zero_grad()
    #get input, labels, lengths for packing
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    class_inp = one_hot(labels[train_track[k*batch_size:(k+1)*batch_size]],2).to(device)
    #labels for rec error
    oh = inp.long().to(device)
    #pass through network
    gen,kl = net(inp,class_inp,les)
    #record average loss over nonzero losses
    rec_loss = loss_func(gen,oh[:,1:mlen])
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    klp += kl.item()
    #backpropagation
    loss = torch.mean(loss_func(gen,oh[:,1:mlen])) + kl*klcoef[i]
    loss.backward()
    #gradient clip
    torch.nn.utils.clip_grad_norm_(net.parameters(),max_norm=1)
    optimizer.step()
    k += 1
  trainloss.append(lp/len(input))
  trainkl.append(klp/len(input))
  #Calculate loss over validation set
  with torch.no_grad():
    k = 0
    while k < len(vind)//batch_size + 1:
      inp = validation[valid_track[k*batch_size:(k+1)*batch_size]]
      les = vlengs[valid_track[k*batch_size:(k+1)*batch_size]]
      oh = inp.long().to(device)
      class_inp = one_hot(vlabels[valid_track[k*batch_size:(k+1)*batch_size]],2).to(device)
      gen,kl = net(inp,class_inp,les)
      rec_loss = loss_func(gen,oh[:,1:mlen])
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      klp += kl.item()
      k += 1
    validloss.append(lp/len(validation))
    validkl.append(klp/len(validation))
  print(i,trainloss[-1],trainkl[-1],validloss[-1],validkl[-1])
  i += 1

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn.functional import softmax
device = torch.device('cuda:0')
#VAE with LSTM and Attention,style as Linear Layer
class LSTM_ATN_VAE(nn.Module):
  def __init__(self, input_length, hidden_z,num_classes, out_length, attn_length, layers,vocab_length, gen_length,drop=0.1,temp=10):
    super(LSTM_ATN_VAE,self).__init__()
    #length of generation
    self.genlen = gen_length
    #embedding dim
    self.inp = input_length
    #vocab length
    self.nvocab = vocab_length
    #length of latent dim
    self.out = out_length
    #hidden length of encoder - nedded for attention
    self.hidz = hidden_z
    #number of layers
    self.layers = layers
    #mean and variance layer
    self.mean_layer = nn.Linear(layers*hidden_z,out_length)
    self.var_layer = nn.Linear(layers*hidden_z,out_length)
    #encoder
    self.enc_z = nn.LSTM(input_length+out_length,hidden_z,layers,batch_first=True)
    #style information linear layer
    self.style_expand = nn.Linear(num_classes,out_length)
    #generator cell - iterated over sequence length - 1 for decoded sequence
    self.dec = nn.LSTMCell(input_length+hidden_z+out_length,out_length)
    #attention linear layer over encoder hidden
    self.attn_enc = nn.Linear(hidden_z,attn_length)
    #attention linear layer over current generator hidden
    self.attn_hid = nn.Linear(out_length,attn_length)
    #reduce to vector
    self.attn_reduce = nn.Linear(attn_length,1)
    #ReLU activation
    self.relu = nn.ReLU()
    #word embeddings
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #dropout
    self.drop = nn.Dropout(p=drop)
    #expand to vocab size
    self.llayer = nn.Linear(out_length,vocab_length)
    #number of layers
    self.numlayers = layers
    #softmax
    self.softmax = nn.Softmax(dim=1)
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #matrix of word embeddings
    x = self.embedding(x)
    #getting style information
    y = self.style_expand(y)
    #feeding packed sequence through encoder, switching back to padded
    ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    out,(hz,_) = self.enc_z(ex)
    out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
    #converts hidden layer to latent mean/logvariance
    hz = hz.view((hz.size()[1],hz.size()[2]*(self.numlayers)))
    mean = self.mean_layer(hz)
    var= self.var_layer(hz)
    #draw from distribution
    draw = torch.normal(torch.zeros(var.size()),torch.ones(var.size())).to(device)
    z = draw*torch.exp(var) + mean
    #calculate KL divergence
    kl = 0.5*(torch.mean(-torch.sum(var,axis=1) + torch.sum(torch.exp(var),axis=1) + torch.sum(torch.square(mean),axis=1)-var.size()[1]))
    #iterating over generation length
    gen = torch.zeros((x.size()[0],self.nvocab,self.genlen)).to(device)
    i = 0
    while i < self.genlen:
      #calculate attention weights
      attn_weights = self.softmax(self.attn_reduce(self.relu(self.attn_enc(out) + self.attn_hid(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
      attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
      #apply weights to encoder hidden state
      attn_vec = torch.bmm(attn_weights,out)
      attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
      #create new input
      inp = torch.cat((attn_vec,y,x[:,i,:]),1)
      #dropout
      inp = self.drop(inp)
      #pass through generator
      z,_ = self.dec(inp)
      #expand to vocab length
      g = self.llayer(z)
      gen[:,:,i] = g
      i += 1
    #rescale with temperature
    gen = (gen - torch.max(gen,1)[0].unsqueeze(1).repeat(1,len(vocab),1))/self.temp
    return gen,kl
    


In [ ]:
#VAE with LSTM and attention, style as linear layer training
#Set seeds for reproducability
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#Choose which labels we want to use - only two at a time
a0 = np.where(ind==0)[0]
a1 = np.where(ind==2)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#Divide data into training and validation sets
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[0] = 0
weights[3] = 0
#initialize network,loss, and optimizer
net = LSTM_ATN_VAE(300,100,2,300,100,1,len(vocab.keys()),mlen-1,0.1,temp=10)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(),lr=0.0001)
batch_size = 256
trainloss = []
trainkl = []
validloss = []
validkl = []
#coefficient for KL annealing
klcoef = np.concatenate((np.zeros(250),np.linspace(0,1,250),np.ones(9500)))
i = 0
while i < 5280:
  #sample random minibatch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #zero gradient
    optimizer.zero_grad()
    #get input, lengths, and class input for minibatch
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    oh = inp.long().to(device)
    class_inp = one_hot(labels[train_track[k*batch_size:(k+1)*batch_size]],2).to(device)
    #pass through network
    gen,kl = net(inp,class_inp,les)
    #record loss
    rec_loss = loss_func(gen,oh[:,1:mlen])
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    klp += kl.item()
    #backward gradient
    loss = torch.mean(loss_func(gen,oh[:,1:mlen])) + kl*klcoef[i]
    loss.backward()
    #clip gradient
    torch.nn.utils.clip_grad_norm_(net.parameters(),max_norm=1)
    optimizer.step()
    k += 1
  trainloss.append(lp/len(input))
  trainkl.append(klp/len(input))
  #find loss over validation set
  with torch.no_grad():
    k = 0
    while k < len(vind)//batch_size + 1:
      inp = validation[valid_track[k*batch_size:(k+1)*batch_size]]
      les = vlengs[valid_track[k*batch_size:(k+1)*batch_size]]
      oh = inp.long().to(device)
      class_inp = one_hot(vlabels[valid_track[k*batch_size:(k+1)*batch_size]],2).to(device)
      gen,kl = net(inp,class_inp,les)
      rec_loss = loss_func(gen,oh[:,1:mlen])
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      klp += kl.item()
      k += 1
    validloss.append(lp/len(validation))
    validkl.append(klp/len(validation))
  print(i,trainloss[-1],trainkl[-1],validloss[-1],validkl[-1])
  i += 1

In [ ]:
#Loading a saved ATN VAE model
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#Choose which labels we want to use - only two at a time
a0 = np.where(ind==0)[0]
a1 = np.where(ind==1)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
#Divide data into training and validation sets
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#Weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[3] = 0
weights[0] = 0
net = LSTM_ATN_VAE(300,100,2,300,100,1,len(vocab.keys()),mlen-1,drop=0.1,temp=10)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
net = net.to(device)
#Change path to where model is saved - make sure labels are consistent with chosen
net.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/Thesis/ATN_VAE_0_1.pt'))
net.eval()
batch_size=256
lp = 0
with torch.no_grad():
  k = 0
  while k < len(vind)//batch_size + 1:
    inp = validation[valid_track[k*batch_size:(k+1)*batch_size]]
    les = vlengs[valid_track[k*batch_size:(k+1)*batch_size]]
    oh = inp.long().to(device)
    class_inp = one_hot(vlabels[valid_track[k*batch_size:(k+1)*batch_size]],2).to(device)
    gen,kl = net(inp,class_inp,les)
    rec_loss = loss_func(gen,oh[:,1:mlen])
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    k += 1
print(lp/len(validation))

In [ ]:
#Base classifier
np.random.seed(100)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
#term frequency matrix
def tf(poems):
  vocab = []
  for p in poems:
    for s in p:
      for st in s: 
        if st not in vocab:
          vocab.append(st)
  mat_tf = np.zeros((len(poems),len(vocab)))
  i = 0
  while i < len(poems):
    for line in poems[i]:
      for word in line:
        mat_tf[i,vocab.index(word)] += 1
    i += 1
  return mat_tf,vocab
def to_mat(p,voc):
  mat = np.zeros((len(p),len(voc)))
  i = 0
  for poem in p:
    for line in poem:
      for word in line:
        if word in voc:
          mat[i,voc.index(word)] += 1
    i += 1
  return mat
mtf,avail_vocab = tf(avail_poems)
plabels = np.array(avail_ind)
#train classifier
X_train,X_test,y_train,y_test = train_test_split(mtf,plabels,test_size=0.33,random_state=100,stratify=plabels)
clf = LinearSVC(random_state=100)
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
print(len(np.where(pred==y_test)[0])/len(y_test))

In [ ]:
#here we generate transferred poems and find style transfer intensity + classification accuracy
#list of vocab
ke = list(vocab.keys())
#beam width for beam search
beam_width = 50
real_poems = []
transferred_poems = []
sample_labels = []
print(len(avail_poems))
#iterate over all available poems
i = 0
while i < len(avail_poems):
  av = avail_poems[i]
  real_poems.append(av)
  sample_labels.append(avail_ind[i])
  genpoem = []
  for line in av:
    if len(line) == 0:
      pass
    else:
      #get acutal label and modified label
      curlab = one_hot([avail_ind[i]],2).to(device)
      modlab = one_hot([1 - avail_ind[i]],2).to(device)
      #generate input as numbers
      leng = []
      a = []
      a.append(vocab['<go>'])
      for h in line:
        if h in vocab:
          a.append(vocab[h])
        else:
          a.append(vocab['<unk>'])
      if len(a) > mlen-1:
        a = a[0:mlen-1]
        a.append(vocab['<eos>'])
        leng.append(mlen)
      else:
        a.append(vocab['<eos>'])
        car = len(a)
        leng.append(car)
        j = 0
        while j < mlen-car:
          a.append(vocab['<pad>'])
          j += 1
      with torch.no_grad():
        #create input matrix
        x = torch.tensor(a).to(device).reshape((1,-1))
        les = torch.tensor(leng)
        #get embedding
        x = net.embedding(x)
        #linear style information for real and transferred label
        y = net.style_expand(curlab)
        mody = net.style_expand(modlab)
        #pass through encoder with real label
        ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
        ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
        out,(hz,_) = net.enc_z(ex)
        out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
        #draw from prior
        z = torch.normal(torch.zeros((x.size()[0],net.out)),torch.ones((x.size()[0],net.out))).to(device)
        gen = []
        #iterate over generation length
        k = 0
        while k < mlen-1:
          #at first step we only have one real input
          if k == 0:
            #create attention vector
            attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn_enc(out) + net.attn_hid(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
            attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
            attn_vec = torch.bmm(attn_weights,out)
            attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
            #create input
            inp = torch.cat((attn_vec,mody,x[:,k,:]),1)
            #pass through decoder
            z,_ = net.dec(inp)
            #expand to vocab size
            ou = net.llayer(z)
            #store hidden state for attention - same for each beam
            z = z.repeat(beam_width,1)
            #store encoder hidden states for attention - same for each beam
            out = out.repeat(beam_width,1,1)
            #get top n with n specified by beam width
            ou = torch.nn.functional.softmax(ou.reshape((-1)),dim=0)
            bind = torch.argsort(ou,descending=True)[0:beam_width]
            #get embeddings
            ip = net.embedding(bind)
            #get probabilities
            prob = ou[bind]
            #store words
            ct = 0
            while ct < len(bind):
              gen.append([ke[bind[ct].cpu().item()]])
              ct += 1
          else:
            #create attention vectors
            attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn_enc(out) + net.attn_hid(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
            attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
            attn_vec = torch.bmm(attn_weights,out)
            attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
            #create input (batch size of beam width)
            inp = torch.cat((attn_vec,mody.repeat(beam_width,1),ip),1)
            #pass through decoder
            z,_ = net.dec(inp)
            #expand to vocab length
            ou = net.llayer(z)
            #get top n by beam width for each beam
            ou = torch.nn.functional.softmax(ou,dim=1)
            aval,aind = torch.sort(ou,dim=1,descending=True)
            aval = aval[:,0:beam_width]
            aind = aind[:,0:beam_width]
            #get probabilities for each overall sequence - multiply current prob by original beam prob
            aval = aval*(prob.unsqueeze(0).repeat(beam_width,1))
            #get top n overall
            aval = aval.reshape((beam_width**2))
            bind = torch.argsort(aval,descending=True)[0:beam_width]
            #store new probabilities
            prob = aval[bind]
            ip = torch.zeros((beam_width,x.size()[2])).to(device)
            ct = 0
            ge = []
            #store word and embedding corresponding to the top n sequences by beam width
            while ct < len(bind):
              xind = bind[ct].cpu().item()//beam_width
              yind = bind[ct].cpu().item()%beam_width
              ip[ct] = net.embedding(aind[xind,yind])
              ge.append(gen[xind] + [ke[aind[xind,yind].cpu().item()]])
              ct += 1
            gen = ge.copy()
          k += 1
      #append list of beam sequences for each generated line
      genpoem.append(gen)
  l = 0
  rp = []
  while l < len(genpoem):
    #choose best generated line and convert to engliosh sentence
    gp = genpoem[l][0]
    rl = []
    for word in gp:
      if word in ['<pad>', '<unk>', '<go>']:
        pass
      elif word == '<eos>':
        break
      else:
        rl.append(word)
    rp.append(rl)
    l += 1
  #append new poem to transferred poem
  transferred_poems.append(rp)
  print(i)
  i += 1
#Calculate style transfer intensity and classification accuracy
i = 0
avchange = 0
better = np.zeros(len(real_poems))
tclass = np.zeros(len(real_poems))
while i < len(real_poems):
  rlab = sample_labels[i]
  tmat = to_mat([real_poems[i]] + [transferred_poems[i]],avail_vocab)
  df = clf.decision_function(tmat)
  pred = clf.predict(tmat)
  if rlab == 0:
    avchange += (df[1] - df[0])/np.abs(df[0])
    if pred[1] == 1:
      tclass[i] = 1
  else:
    avchange += (df[0] - df[1])/np.abs(df[0])
    if pred[1] == 0:
      tclass[i] = 1
  i += 1
print('Style Transfer Intensity')
print(avchange/i)
print('Classification Accuracy')
print(np.sum(tclass)/len(tclass))

In [ ]:
#Can experiment with transferring specific poems
sp = np.array(sample_labels)
tc = np.array(tclass)
print(np.where(tclass==0)[0])
print(np.where(tclass==1)[0])
print(np.where(sp==0)[0])
print(np.where(sp==1)[0])
ind = 92
print(tclass[ind])
print('Real Poem:')
for r in real_poems[ind]:
  st = ''
  for word in r:
    st += word + ' '
  print(st)
print('Transferred Poem:')
for r in transferred_poems[ind]:
  st = ''
  for word in r:
    st += word + ' '
  print(st)

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn.functional import softmax
device = torch.device('cuda:0')
#VAE with bidirectional LSTM and Attention,style as linear layer
class LSTM_ATN_VAE_BD(nn.Module):
  def __init__(self, input_length, hidden_z,num_classes, out_length, attn_length, layers,vocab_length, gen_length,drop=0.1,temp=10):
    super(LSTM_ATN_VAE_BD,self).__init__()
    #generation length
    self.genlen = gen_length
    #input length
    self.inp = input_length
    #vocab length
    self.nvocab = vocab_length
    #latent dim
    self.out = out_length
    #hidden dim of encoder
    self.hidz = hidden_z
    #number of layers
    self.numlayers = layers
    #mean and logvariance layers
    self.mean_layer = nn.Linear(2*layers*hidden_z,out_length)
    self.var_layer = nn.Linear(2*layers*hidden_z,out_length)
    #encoder
    self.enc_z = nn.LSTM(input_length+out_length,hidden_z,layers,batch_first=True,bidirectional=True)
    #linear layer for style
    self.style_expand = nn.Linear(num_classes,out_length)
    #generator
    self.dec = nn.LSTMCell(input_length+2*hidden_z+out_length,out_length)
    #attention linear layer for encoder hidden
    self.attn_enc = nn.Linear(2*hidden_z,attn_length)
    #attention linear layer for generator hidden
    self.attn_hid = nn.Linear(out_length,attn_length)
    #convert to vector
    self.attn_reduce = nn.Linear(attn_length,1)
    #dropout
    self.drop = nn.Dropout(p=drop)
    #ReLU activation
    self.relu = nn.ReLU()
    #word emebedding
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #softmax layer
    self.softmax = nn.Softmax(dim=1)
    #linear layer to expand to vocab length
    self.llayer = nn.Linear(out_length,vocab_length)
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #get word embeddings
    x = self.embedding(x)
    #expand for style information
    y = self.style_expand(y)
    #pass packed sequence through encoder and unpack
    ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    out,(hz,_) = self.enc_z(ex)
    out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
    #get mean and log variance of latent
    hz = hz.view((hz.size()[1],hz.size()[2]*2*(self.numlayers)))
    mean = self.mean_layer(hz)
    var= self.var_layer(hz)
    #draw from distribution with reparametrization trick
    draw = torch.normal(torch.zeros(var.size()),torch.ones(var.size())).to(device)
    z = draw*torch.exp(var) + mean
    #get KL divergence
    kl = 0.5*(torch.mean(-torch.sum(var,axis=1) + torch.sum(torch.exp(var),axis=1) + torch.sum(torch.square(mean),axis=1)-var.size()[1]))
    gen = torch.zeros((x.size()[0],self.nvocab,self.genlen)).to(device)
    #iterate over gen length
    i = 0
    while i < self.genlen:
      #create attention weights
      attn_weights = self.softmax(self.attn_reduce(self.relu(self.attn_enc(out) + self.attn_hid(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
      attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
      #apply to encoder states
      attn_vec = torch.bmm(attn_weights,out)
      attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
      #create input
      inp = torch.cat((attn_vec,y,x[:,i,:]),1)
      #dropout
      inp = self.drop(inp)
      #pass through generator
      z,_ = self.dec(inp)
      #expand to vocab
      g = self.llayer(z)
      gen[:,:,i] = g
      i += 1
    #rescale to temperature
    gen = (gen - torch.max(gen,1)[0].unsqueeze(1).repeat(1,self.nvocab,1))/self.temp
    return gen,kl
    


In [ ]:
#VAE with LSTM, style as linear layer training
#Set seeds for reproducability
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#choose which indices to use - two at a time
a0 = np.where(ind==0)[0]
a1 = np.where(ind==1)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#divide into training and validaiton
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#Weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[3] = 0
weights[0] = 0
#initialize network,loss,optimizer
net = LSTM_ATN_VAE_BD(300,100,2,300,100,1,len(vocab.keys()),mlen-1,drop=0.1,temp=10)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
net = net.to(device)
optimizer = torch.optim.Adam(net.parameters(),lr=0.0001)
batch_size = 512
trainloss = []
trainkl = []
validloss = []
validkl = []
#coefficients for KL annealing
klcoef = np.concatenate((np.zeros(250),np.linspace(0,1,250),np.ones(9500)))
i = 0
while i < 4924:
  #sample random minibatch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #zero gradient
    optimizer.zero_grad()
    #get inputs
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    oh = inp.long().to(device)
    class_inp = one_hot(labels[train_track[k*batch_size:(k+1)*batch_size]],2).to(device)
    #pass through network
    gen,kl = net(inp,class_inp,les)
    #record loss
    rec_loss = loss_func(gen,oh[:,1:mlen])
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    klp += kl.item()
    #backward gradient
    loss = torch.mean(loss_func(gen,oh[:,1:mlen])) + kl*klcoef[i]
    loss.backward()
    #clip gradient
    torch.nn.utils.clip_grad_norm_(net.parameters(),max_norm=1)
    optimizer.step()
    k += 1
  trainloss.append(lp/len(input))
  trainkl.append(klp/len(input))
  #get validation loss
  with torch.no_grad():
    k = 0
    while k < len(vind)//batch_size + 1:
      inp = validation[valid_track[k*batch_size:(k+1)*batch_size]]
      les = vlengs[valid_track[k*batch_size:(k+1)*batch_size]]
      oh = inp.long().to(device)
      class_inp = one_hot(vlabels[valid_track[k*batch_size:(k+1)*batch_size]],2).to(device)
      gen,kl = net(inp,class_inp,les)
      rec_loss = loss_func(gen,oh[:,1:mlen])
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      klp += kl.item()
      k += 1
    validloss.append(lp/len(validation))
    validkl.append(klp/len(validation))
    print(i,trainloss[-1],trainkl[-1],validloss[-1],validkl[-1])
    i += 1

In [ ]:
!pip3 install py-rouge

In [ ]:
#getting BLEU and ROUGE scores 
import nltk
import rouge
nltk.download('punkt')
apply_avg = True
apply_best = False
evaluator = rouge.Rouge(metrics=['rouge-n'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
bleu_tref = []
bleu_thyp = []
bleu_vref = []
bleu_vhyp = []
rouge_tref = []
rouge_thyp = []
rouge_vref = []
rouge_vhyp = []
ke = list(vocab.keys())
k = 0
while k < len(input):
  l = []
  for j in input[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_tref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_tref.append(rs)
  with torch.no_grad():
    out,_ = net(input[k:k+1],one_hot(labels[k:k+1],2).to(device),lengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_thyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_thyp.append(rs)
  k += 1
k = 0
while k < len(validation):
  l = []
  for j in validation[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_vref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_vref.append(rs)
  with torch.no_grad():
    out,_ = net(validation[k:k+1],one_hot(vlabels[k:k+1],2).to(device),vlengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_vhyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_vhyp.append(rs)
  k += 1
tscores = evaluator.get_scores(rouge_tref, rouge_thyp)
vscores = evaluator.get_scores(rouge_vref, rouge_vhyp)
print('BLEU Score Training')
print(nltk.translate.bleu_score.corpus_bleu(bleu_tref,bleu_thyp))
print('BLEU Score Validation')
print(nltk.translate.bleu_score.corpus_bleu(bleu_vref,bleu_vhyp))
print('ROUGE-1 Training')
print(tscores['rouge-1']['f'])
print('ROUGE-2 Training')
print(tscores['rouge-2']['f'])
print('ROUGE-1 Validation')
print(vscores['rouge-1']['f'])
print('ROUGE-2 Validation')
print(vscores['rouge-2']['f'])


In [ ]:
#Code to load saved ATN VAE BD Model
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#Choose which labels we want to use - only two at a time
a0 = np.where(ind==2)[0]
a1 = np.where(ind==3)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
#Divide data into training and validation sets
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#Weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[3] = 0
weights[0] = 0
net = LSTM_ATN_VAE_BD(300,100,2,300,100,1,len(vocab.keys()),mlen-1,drop=0.1,temp=10)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
net = net.to(device)
#change path to where model is saved. Make sure numbers match with chosen labels
net.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/Thesis/ATN_VAE_BD_2_3.pt'))
net.eval()
batch_size=256
lp = 0
with torch.no_grad():
  k = 0
  while k < len(vind)//batch_size + 1:
    inp = validation[valid_track[k*batch_size:(k+1)*batch_size]]
    les = vlengs[valid_track[k*batch_size:(k+1)*batch_size]]
    oh = inp.long().to(device)
    class_inp = one_hot(vlabels[valid_track[k*batch_size:(k+1)*batch_size]],2).to(device)
    gen,kl = net(inp,class_inp,les)
    rec_loss = loss_func(gen,oh[:,1:mlen])
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    k += 1
print(lp/len(validation))

In [ ]:
#Base classifier
np.random.seed(100)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
#term frequency matrix
def tf(poems):
  vocab = []
  for p in poems:
    for s in p:
      for st in s: 
        if st not in vocab:
          vocab.append(st)
  mat_tf = np.zeros((len(poems),len(vocab)))
  i = 0
  while i < len(poems):
    for line in poems[i]:
      for word in line:
        mat_tf[i,vocab.index(word)] += 1
    i += 1
  return mat_tf,vocab
def to_mat(p,voc):
  mat = np.zeros((len(p),len(voc)))
  i = 0
  for poem in p:
    for line in poem:
      for word in line:
        if word in voc:
          mat[i,voc.index(word)] += 1
    i += 1
  return mat
mtf,avail_vocab = tf(avail_poems)
plabels = np.array(avail_ind)
#train classifier
X_train,X_test,y_train,y_test = train_test_split(mtf,plabels,test_size=0.33,random_state=100,stratify=plabels)
clf = LinearSVC(random_state=100)
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
print(len(np.where(pred==y_test)[0])/len(y_test))

In [ ]:
#Refer to ATN VAE beam search - this is the same but adjusted for ATN BD VAE
ke = list(vocab.keys())
beam_width = 10
real_poems = []
transferred_poems = []
sample_labels = []
print(len(avail_poems))
i = 0
while i < len(avail_poems):
  av = avail_poems[i]
  real_poems.append(av)
  sample_labels.append(avail_ind[i])
  genpoem = []
  for line in av:
    if len(line) == 0:
      pass
    else:
      curlab = one_hot([avail_ind[i]],2).to(device)
      modlab = one_hot([1 - avail_ind[i]],2).to(device)
      leng = []
      a = []
      a.append(vocab['<go>'])
      for h in line:
        if h in vocab:
          a.append(vocab[h])
        else:
          a.append(vocab['<unk>'])
      if len(a) > mlen-1:
        a = a[0:mlen-1]
        a.append(vocab['<eos>'])
        leng.append(mlen)
      else:
        a.append(vocab['<eos>'])
        car = len(a)
        leng.append(car)
        j = 0
        while j < mlen-car:
          a.append(vocab['<pad>'])
          j += 1
      with torch.no_grad():
        x = torch.tensor(a).to(device).reshape((1,-1))
        les = torch.tensor(leng)
        x = net.embedding(x)
        y = net.style_expand(curlab)
        mody = net.style_expand(modlab)
        ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
        ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
        out,(hz,_) = net.enc_z(ex)
        out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
        z = torch.normal(torch.zeros((x.size()[0],net.out)),torch.ones((x.size()[0],net.out))).to(device)
        best_embed = net.embedding(torch.tensor([2]).to(device))
        gen = []
        k = 0
        while k < mlen-1:
          if k == 0:
            attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn_enc(out) + net.attn_hid(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
            attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
            attn_vec = torch.bmm(attn_weights,out)
            attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
            inp = torch.cat((attn_vec,mody,x[:,k,:]),1)
            z,_ = net.dec(inp)
            ou = net.llayer(z)
            z = z.repeat(beam_width,1)
            out = out.repeat(beam_width,1,1)
            ou = torch.nn.functional.softmax(ou.reshape((-1)),dim=0)
            bind = torch.argsort(ou,descending=True)[0:beam_width]
            ip = net.embedding(bind)
            prob = ou[bind]
            ct = 0
            while ct < len(bind):
              gen.append([ke[bind[ct].cpu().item()]])
              ct += 1
          else:
            attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn_enc(out) + net.attn_hid(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
            attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
            attn_vec = torch.bmm(attn_weights,out)
            attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
            inp = torch.cat((attn_vec,mody.repeat(beam_width,1),ip),1)
            z,_ = net.dec(inp)
            ou = net.llayer(z)
            ou = torch.nn.functional.softmax(ou,dim=1)
            aval,aind = torch.sort(ou,dim=1,descending=True)
            aval = aval[:,0:beam_width]
            aind = aind[:,0:beam_width]
            aval = aval*(prob.unsqueeze(0).repeat(beam_width,1))
            aval = aval.reshape((beam_width**2))
            bind = torch.argsort(aval,descending=True)[0:beam_width]
            prob = aval[bind]
            ip = torch.zeros((beam_width,x.size()[2])).to(device)
            ct = 0
            ge = []
            while ct < len(bind):
              xind = bind[ct].cpu().item()//beam_width
              yind = bind[ct].cpu().item()%beam_width
              ip[ct] = net.embedding(aind[xind,yind])
              ge.append(gen[xind] + [ke[aind[xind,yind].cpu().item()]])
              ct += 1
            gen = ge.copy()
          k += 1
      genpoem.append(gen)
  l = 0
  rp = []
  while l < len(genpoem):
    gp = genpoem[l][0]
    rl = []
    for word in gp:
      if word in ['<pad>', '<unk>', '<go>']:
        pass
      elif word == '<eos>':
        break
      else:
        rl.append(word)
    rp.append(rl)
    l += 1
  transferred_poems.append(rp)
  print(i)
  i += 1
i = 0
avchange = 0
better = np.zeros(len(real_poems))
tclass = np.zeros(len(real_poems))
while i < len(real_poems):
  rlab = sample_labels[i]
  tmat = to_mat([real_poems[i]] + [transferred_poems[i]],avail_vocab)
  df = clf.decision_function(tmat)
  pred = clf.predict(tmat)
  if rlab == 0:
    avchange += (df[1] - df[0])/np.abs(df[0])
    if pred[1] == 1:
      tclass[i] = 1
  else:
    avchange += (df[0] - df[1])/np.abs(df[0])
    if pred[1] == 0:
      tclass[i] = 1
  i += 1
print('Style Transfer Intensity')
print(avchange/i)
print('Classification Accuracy')
print(np.sum(tclass)/len(tclass))

In [ ]:
#try transferring various poems
sp = np.array(sample_labels)
tc = np.array(tclass)
print(np.where(tclass==0)[0])
print(np.where(tclass==1)[0])
print(np.where(sp==0)[0])
print(np.where(sp==1)[0])
ind = 77
print(tclass[ind])
print('Real Poem:')
for r in real_poems[ind]:
  st = ''
  for word in r:
    st += word + ' '
  print(st)
print('Transferred Poem:')
for r in transferred_poems[ind]:
  st = ''
  for word in r:
    st += word + ' '
  print(st)

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
device = torch.device('cuda:0')
#Cross aligned AE with LSTM, style as linear layer 
class LSTM_CAE(nn.Module):
  def __init__(self, input_length, num_classes, hidden_length,layers,vocab_length, gen_length,temp):
    super(LSTM_CAE,self).__init__()
    #number of classes
    self.numclasses = num_classes
    #style information layer
    self.style_layer = nn.Linear(num_classes,hidden_length)
    #generation length
    self.genlen = gen_length
    #input length
    self.inp = input_length
    #vocab length
    self.nvocab = vocab_length
    #encoder
    self.enc = nn.LSTM(input_length+hidden_length,hidden_length,layers,batch_first=True)
    #generator
    self.dec = nn.LSTMCell(input_length+2*hidden_length,input_length)
    #word embedding
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #linear to expand to vocab size
    self.llayer = nn.Linear(input_length,vocab_length)
    #number of layers
    self.numlayers = layers
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #real labels
    oy = self.style_layer(one_hot(y,2).to(device))
    #transferred labels
    mody = self.style_layer(one_hot(1-y,2).to(device))
    #get word embeddings
    x = self.embedding(x)
    #pass through encoder
    ex = torch.cat((oy.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    _,(henc,_) = self.enc(ex)
    henc = henc.view((henc.size()[1],henc.size()[2]*self.numlayers))
    #real and transferred hidden states - for discriminator
    gen_fake = torch.zeros((x.size()[0],self.genlen,self.inp)).to(device)
    gen_real = torch.zeros((x.size()[0],self.genlen,self.inp)).to(device)
    #generated words for reconstruction loss
    gen = torch.zeros((x.size()[0],self.nvocab,self.genlen)).to(device)
    #iterate over generation length
    i = 0
    while i < self.genlen:
      #real input
      inp1 = torch.cat((henc,oy,x[:,i,:]),1)
      #transferred input
      if i == 0:
        inp2 = torch.cat((henc,mody,x[:,0,:]),1)
      else:
        inp2 = torch.cat((henc,mody,all_embed),1)
      #pass through generator
      ip = torch.cat((inp1,inp2),0)
      z,_ = self.dec(ip)
      #store hidden states for discriminator
      cut = int(len(z)/2)
      gen_real[:,i,:] = z[0:cut]
      gen_fake[:,i,:] = z[cut:len(z)]
      #expand to vocab length
      kp = self.llayer(z)
      #rescale
      kp = kp - torch.max(kp,dim=1)[0].unsqueeze(1).repeat(1,kp.size()[1])
      #store distribution over vocab for reconstruction
      gen[:,:,i] = kp[0:cut]
      #get word output for transferred samples as described in paper - weighted average over distribution
      su = torch.sum(torch.exp(kp[cut:len(kp)]/self.temp),axis=1)
      softmax = (torch.exp(kp[cut:len(kp)]/self.temp).T/(su)).T
      ta = torch.arange(self.nvocab).to(device)
      all_embed = self.embedding(ta)
      all_embed = softmax@all_embed
      i += 1
    return gen,gen_real,gen_fake
#discriminator
class Disc(nn.Module):
  def __init__(self, input_length,disc_hidden_length):
    super(Disc,self).__init__()
    #bidirectional LSTM
    self.disc = nn.LSTM(input_length,disc_hidden_length,batch_first=True,bidirectional=True)
    #linear layer to reduce - two dimensions with 1=real,0=fake
    self.disc_linear = nn.Linear(disc_hidden_length*2,2)
  def forward(self,disc_store):
    _,(hid,_) = self.disc(disc_store)
    hid = hid.reshape((hid.size()[1],hid.size()[2]*2))
    hid = self.disc_linear(hid)
    return hid


In [ ]:
#CAE with LSTM, style as linear layer training
#Set seeds for deterministic 
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
#choose labels we want - two at a time
ind = np.array(author_labels)
a0 = np.where(ind==0)[0]
a1 = np.where(ind==1)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#divide into training and validation sets
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#Weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[3] = 0
weights[0] = 0
#initialize network and discriminator for 0 and 1, loss, and three separate optimizers
net = LSTM_CAE(300,2,300,1,len(vocab.keys()),mlen-1,10)
disc_0 = Disc(300,100)
disc_1 = Disc(300,100)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
lf2 = nn.CrossEntropyLoss(reduction='mean')
net = net.to(device)
disc_0 = disc_0.to(device)
disc_1 = disc_1.to(device)
opt1 = torch.optim.Adam(net.parameters(),lr=0.0001)
opt2 = torch.optim.Adam(disc_0.parameters(),lr=0.0001)
opt3 = torch.optim.Adam(disc_1.parameters(),lr=0.0001)
optimizer = [opt3,opt2,opt1]
models = [disc_0,disc_1,net]
trainloss = []
traindisc = []
valloss = []
td = []
batch_size=512
i = 0
while i < 3000:
  #sample random minibatch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #get inputs
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    oh = inp[:,1:mlen].long().to(device)
    curlab = labels[train_track[k*batch_size:(k+1)*batch_size]]
    #pass through network
    gen,gen_real,gen_fake = net(inp,curlab,les)
    #figure out where labels are 0 and 1
    zero_labels = torch.where(curlab==0)[0]
    one_labels = torch.where(curlab==1)[0]
    #Three possibilities - only 0,only 1, both
    if len(zero_labels)==0 and len(one_labels) > 0:
      #discriminator outputs
      out0 = disc_0(gen_real)
      out1 = disc_1(gen_fake)
      #labels for discriminators
      lab0 = torch.ones(len(gen_real)).long().to(device)
      lab1 = torch.zeros(len(gen_fake)).long().to(device)
      #output to check how well fake fools discriminator
      zero_g = None
      one_g = disc_1(gen_fake)
      #labels for fake generator
      lab0_g = None
      lab1_g = torch.ones(len(gen_fake)).long().to(device)
    elif len(one_labels)==0 and len(zero_labels) > 0:
      #discriminator outputs
      out0 = disc_0(gen_fake)
      out1 = disc_1(gen_real)
      #labels for discriminators
      lab0 = torch.ones(len(gen_fake)).long().to(device)
      lab1 = torch.zeros(len(gen_real)).long().to(device)
      #output to check how well fake fools discriminator
      zero_g = disc_0(gen_fake)
      one_g = None
      #labels for fake generator
      lab1_g = None
      lab0_g = torch.ones(len(gen_fake)).long().to(device)
    else:
      #all inputs for zero discriminator
      g0 = torch.cat((gen_real[zero_labels],gen_fake[one_labels]),0)
      #all inputs for one discriminator
      g1 = torch.cat((gen_real[one_labels],gen_fake[zero_labels]),0)
      #discriminator outputs
      out0 = disc_0(g0)
      out1 = disc_1(g1)
      #discriminator labels
      lab0 = torch.cat((torch.ones(len(zero_labels)),torch.zeros(len(one_labels))),0).long().to(device)
      lab1 = torch.cat((torch.ones(len(one_labels)),torch.zeros(len(zero_labels))),0).long().to(device)
      #output to check how well fake fools discriminator
      zero_g = disc_0(gen_fake[one_labels])
      one_g = disc_1(gen_fake[zero_labels])
      #labels for fake generator
      lab0_g = torch.ones(len(one_labels)).long().to(device)
      lab1_g = torch.ones(len(zero_labels)).long().to(device)
    #calculate loss of discriminators
    l0 = lf2(out0,lab0)
    l1 = lf2(out1,lab1)
    #calculate adversarial loss - does generator fool discriminator
    if zero_g is not None:
      l0_g = lf2(zero_g,lab0_g)
    else:
      l0_g = 0
    if one_g is not None:
      l1_g = lf2(one_g,lab1_g)
    else:
      l1_g = 0
    #calculate reconstruction loss
    rec_loss = loss_func(gen,oh)
    loss = torch.mean(rec_loss) + (l0_g + l1_g)
    losses = [l0,l1,loss]
    #backward gradient over three networks
    g = 0
    while g < 3:
      optimizer[g].zero_grad()
      losses[g].backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm_(models[g].parameters(),max_norm=1)
      g += 1
    g = 0
    while g < 3:
      optimizer[g].step()
      g += 1
    #record loss
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    k += 1
  trainloss.append(lp/len(input))
  #measure validation loss - here we generate without cross-alignment
  with torch.no_grad():
    k = 0
    lp = 0
    while k < len(validation)//256:
      inp = validation[k*256:(k+1)*256]
      oh = inp[:,1:mlen].long()
      class_inp = one_hot(vlabels[k*256:(k+1)*256],2).to(device)
      le = vlengs[k*256:(k+1)*256]
      y = net.style_layer(class_inp)
      x = net.embedding(inp)
      ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
      ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
      _,(henc,_) = net.enc(ex)
      henc = henc.view((henc.size()[1],henc.size()[2]*net.numlayers))
      gen = torch.zeros((henc.size()[0],net.nvocab,net.genlen)).to(device)
      p = 0
      while p < net.genlen:
        ip = torch.cat((henc,y,x[:,p,:]),1)
        kp,_ = net.dec(ip)
        kp = net.llayer(kp)
        gen[:,:,p] = kp
        p += 1
      rec_loss = loss_func(gen,oh)
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      k += 1
  valloss.append(lp/len(validation))  
  print(i,trainloss[-1],valloss[-1])
  i += 1

In [ ]:
!pip install py-rouge

In [ ]:
#evaluating BLEU and ROUGE - difference here is instead of using original network we have to manually compute (similar to validation loss)
import nltk
import rouge
nltk.download('punkt')
apply_avg = True
apply_best = False
evaluator = rouge.Rouge(metrics=['rouge-n'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
def eval_inp(inp,class_inp,le):
  y = net.style_layer(class_inp)
  x = net.embedding(inp)
  ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
  ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
  _,(henc,_) = net.enc(ex)
  henc = henc.view((henc.size()[1],henc.size()[2]*net.numlayers))
  gen = torch.zeros((henc.size()[0],net.nvocab,net.genlen)).to(device)
  p = 0
  while p < net.genlen:
    ip = torch.cat((henc,y,x[:,p,:]),1)
    kp,_ = net.dec(ip)
    kp = net.llayer(kp)
    gen[:,:,p] = kp
    p += 1
  return gen
bleu_tref = []
bleu_thyp = []
bleu_vref = []
bleu_vhyp = []
rouge_tref = []
rouge_thyp = []
rouge_vref = []
rouge_vhyp = []
ke = list(vocab.keys())
k = 0
while k < len(input):
  l = []
  for j in input[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_tref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_tref.append(rs)
  with torch.no_grad():
    out = eval_inp(input[k:k+1],one_hot(labels[k:k+1],2).to(device),lengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_thyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_thyp.append(rs)
  k += 1
k = 0
while k < len(validation):
  l = []
  for j in validation[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_vref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_vref.append(rs)
  out = eval_inp(validation[k:k+1],one_hot(vlabels[k:k+1],2).to(device),vlengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_vhyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_vhyp.append(rs)
  k += 1
tscores = evaluator.get_scores(rouge_tref, rouge_thyp)
vscores = evaluator.get_scores(rouge_vref, rouge_vhyp)
print('BLEU Score Training')
print(nltk.translate.bleu_score.corpus_bleu(bleu_tref,bleu_thyp))
print('BLEU Score Validation')
print(nltk.translate.bleu_score.corpus_bleu(bleu_vref,bleu_vhyp))
print('ROUGE-1 Training')
print(tscores['rouge-1']['f'])
print('ROUGE-2 Training')
print(tscores['rouge-2']['f'])
print('ROUGE-1 Validation')
print(vscores['rouge-1']['f'])
print('ROUGE-2 Validation')
print(vscores['rouge-2']['f'])

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
device = torch.device('cuda:0')
#Cross aligned AE with Bidirectional LSTM, style as linear layer 
class LSTM_CAE_BD(nn.Module):
  def __init__(self, input_length, num_classes, hidden_length,layers,vocab_length, gen_length,temp):
    super(LSTM_CAE_BD,self).__init__()
    #number of classes
    self.numclasses = num_classes
    #linear for style
    self.style_layer = nn.Linear(num_classes,hidden_length)
    #generation length
    self.genlen = gen_length
    #embedding dim
    self.inp = input_length
    #vocab length
    self.nvocab = vocab_length
    #encoder
    self.enc = nn.LSTM(input_length+hidden_length,hidden_length,layers,batch_first=True,bidirectional=True)
    #generator
    self.dec = nn.LSTMCell(input_length+3*hidden_length,input_length)
    #word embedding
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #linear to expand to vocab length
    self.llayer = nn.Linear(input_length,vocab_length)
    #number of layers
    self.numlayers = layers
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #real labels
    oy = self.style_layer(one_hot(y,2).to(device))
    #transferred labels
    mody = self.style_layer(one_hot(1-y,2).to(device))
    #get word embeddings
    x = self.embedding(x)
    #pass through encoder
    ex = torch.cat((oy.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    _,(henc,_) = self.enc(ex)
    henc = henc.view((henc.size()[1],henc.size()[2]*2*self.numlayers))
    #inputs to discriminator - transferred and real
    gen_fake = torch.zeros((x.size()[0],self.genlen,self.inp)).to(device)
    gen_real = torch.zeros((x.size()[0],self.genlen,self.inp)).to(device)
    #distribution over vocab length for reconstruction loss
    gen = torch.zeros((x.size()[0],self.nvocab,self.genlen)).to(device)
    i = 0
    while i < self.genlen:
      #real input
      inp1 = torch.cat((henc,oy,x[:,i,:]),1)
      #transferred input
      if i == 0:
        inp2 = torch.cat((henc,mody,x[:,0,:]),1)
      else:
        inp2 = torch.cat((henc,mody,all_embed),1)
      #pass through generator
      ip = torch.cat((inp1,inp2),0)
      z,_ = self.dec(ip)
      #store states for discriminator
      cut = int(len(z)/2)
      gen_real[:,i,:] = z[0:cut]
      gen_fake[:,i,:] = z[cut:len(z)]
      #expand to vocab length
      kp = self.llayer(z)
      #rescale
      kp = (kp - torch.max(kp,dim=1)[0].unsqueeze(1).repeat(1,kp.size()[1]))
      #store distribution over vocab
      gen[:,:,i] = kp[0:cut]
      #get words for transferred - weighted average over distribution
      su = torch.sum(torch.exp(kp[cut:len(kp)]/self.temp),axis=1)
      softmax = (torch.exp(kp[cut:len(kp)]/self.temp).T/(su)).T
      ta = torch.arange(self.nvocab).to(device)
      all_embed = self.embedding(ta)
      all_embed = softmax@all_embed
      i += 1
    return gen,gen_fake,gen_real
#discriminator
class Disc(nn.Module):
  def __init__(self, input_length,disc_hidden_length):
    super(Disc,self).__init__()
    #bidirectional LSTM
    self.disc = nn.LSTM(input_length,disc_hidden_length,batch_first=True,bidirectional=True)
    #linear layer to get 2d output - 1=real,0=transferred
    self.disc_linear = nn.Linear(disc_hidden_length*2,2)
  def forward(self,disc_store):
    _,(hid,_) = self.disc(disc_store)
    hid = hid.reshape((hid.size()[1],hid.size()[2]*2))
    hid = self.disc_linear(hid)
    return hid


In [ ]:
#CAE with bidirectional LSTM, style as linear layer training
#set seeds for reproducibility
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#choose which labels to use - two at a time
a0 = np.where(ind==0)[0]
a1 = np.where(ind==1)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#divide into training and validation
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[3] = 0
weights[0] = 0
#initialize network, discriminators, losses, and optimizer
net = LSTM_CAE_BD(300,2,300,1,len(vocab.keys()),mlen-1,10)
disc_0 = Disc(300,100)
disc_1 = Disc(300,100)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
lf2 = nn.CrossEntropyLoss(reduction='mean')
net = net.to(device)
disc_0 = disc_0.to(device)
disc_1 = disc_1.to(device)
opt1 = torch.optim.Adam(net.parameters(),lr=0.0001)
opt2 = torch.optim.Adam(disc_0.parameters(),lr=0.0001)
opt3 = torch.optim.Adam(disc_1.parameters(),lr=0.0001)
optimizer = [opt3,opt2,opt1]
models = [disc_0,disc_1,net]
trainloss = []
traindisc = []
valloss = []
td = []
batch_size=256
i = 0
while i < 1500:
  #sample ranodm minibatch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #gets inputs
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    oh = inp[:,1:mlen].long().to(device)
    curlab = labels[train_track[k*batch_size:(k+1)*batch_size]]
    #pass through networks
    gen,gen_real,gen_fake = net(inp,curlab,les)
    #find where 0 and 1 labels are
    zero_labels = torch.where(curlab==0)[0]
    one_labels = torch.where(curlab==1)[0]
    #three possibilities - all zeros, all ones, both
    if len(zero_labels)==0 and len(one_labels) > 0:
      #discriminator output
      out0 = disc_0(gen_real)
      out1 = disc_1(gen_fake)
      #discriminator labels
      lab0 = torch.ones(len(gen_real)).long().to(device)
      lab1 = torch.zeros(len(gen_fake)).long().to(device)
      #output to check how well fake fools discriminator
      zero_g = None
      one_g = disc_1(gen_fake)
      #fake labels
      lab0_g = None
      lab1_g = torch.ones(len(gen_fake)).long().to(device)
    elif len(one_labels)==0 and len(zero_labels) > 0:
      #discriminator output
      out0 = disc_0(gen_fake)
      out1 = disc_1(gen_real)
      #discriminator labels
      lab0 = torch.ones(len(gen_fake)).long().to(device)
      lab1 = torch.zeros(len(gen_real)).long().to(device)
      #output to check how well fake fools discriminator
      zero_g = disc_0(gen_fake)
      one_g = None
      #fake labels
      lab1_g = None
      lab0_g = torch.ones(len(gen_fake)).long().to(device)
    else:
      #all zero samples
      g0 = torch.cat((gen_real[zero_labels],gen_fake[one_labels]),0)
      #all one samples
      g1 = torch.cat((gen_real[one_labels],gen_fake[zero_labels]),0)
      #discriminator outputs
      out0 = disc_0(g0)
      out1 = disc_1(g1)
      #discriminator labels
      lab0 = torch.cat((torch.ones(len(zero_labels)),torch.zeros(len(one_labels))),0).long().to(device)
      lab1 = torch.cat((torch.ones(len(one_labels)),torch.zeros(len(zero_labels))),0).long().to(device)
      #output to check how well fake fools discriminator
      zero_g = disc_0(gen_fake[one_labels])
      one_g = disc_1(gen_fake[zero_labels])
      #fake labels
      lab0_g = torch.ones(len(one_labels)).long().to(device)
      lab1_g = torch.ones(len(zero_labels)).long().to(device)
    #discrimminator loss
    l0 = lf2(out0,lab0)
    l1 = lf2(out1,lab1)
    #loss for fooling discriminator
    if zero_g is not None:
      l0_g = lf2(zero_g,lab0_g)
    else:
      l0_g = 0
    if one_g is not None:
      l1_g = lf2(one_g,lab1_g)
    else:
      l1_g = 0
    #reconstruction loss
    rec_loss = loss_func(gen,oh)
    loss = torch.mean(rec_loss) + (l0_g + l1_g)
    losses = [l0,l1,loss]
    #backward all gradients
    g = 0
    while g < 3:
      optimizer[g].zero_grad()
      losses[g].backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm_(models[g].parameters(),max_norm=1)
      g += 1
    g = 0
    while g < 3:
      optimizer[g].step()
      g += 1
    #record loss
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    k += 1
  #check validation loss
  trainloss.append(lp/len(input))
  with torch.no_grad():
    k = 0
    lp = 0
    while k < len(validation)//256:
      inp = validation[k*256:(k+1)*256]
      oh = inp[:,1:mlen].long()
      class_inp = one_hot(vlabels[k*256:(k+1)*256],2).to(device)
      le = vlengs[k*256:(k+1)*256]
      y = net.style_layer(class_inp)
      x = net.embedding(inp)
      ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
      ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
      _,(henc,_) = net.enc(ex)
      henc = henc.view((henc.size()[1],henc.size()[2]*2*net.numlayers))
      gen = torch.zeros((henc.size()[0],net.nvocab,net.genlen)).to(device)
      p = 0
      while p < net.genlen:
        ip = torch.cat((henc,y,x[:,p,:]),1)
        kp,_ = net.dec(ip)
        kp = net.llayer(kp)
        gen[:,:,p] = kp
        p += 1
      rec_loss = loss_func(gen,oh)
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      k += 1
  valloss.append(lp/len(validation))  
  print(i,trainloss[-1],valloss[-1])
  i += 1

In [ ]:
#calculate BLEU and ROUGE
import nltk
import rouge
nltk.download('punkt')
apply_avg = True
apply_best = False
evaluator = rouge.Rouge(metrics=['rouge-n'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
def eval_inp(inp,class_inp,le):
  y = net.style_layer(class_inp)
  x = net.embedding(inp)
  ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
  ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
  _,(henc,_) = net.enc(ex)
  henc = henc.view((henc.size()[1],henc.size()[2]*2*net.numlayers))
  gen = torch.zeros((henc.size()[0],net.nvocab,net.genlen)).to(device)
  p = 0
  while p < net.genlen:
    ip = torch.cat((henc,y,x[:,p,:]),1)
    kp,_ = net.dec(ip)
    kp = net.llayer(kp)
    gen[:,:,p] = kp
    p += 1
  return gen
bleu_tref = []
bleu_thyp = []
bleu_vref = []
bleu_vhyp = []
rouge_tref = []
rouge_thyp = []
rouge_vref = []
rouge_vhyp = []
ke = list(vocab.keys())
k = 0
while k < len(input):
  l = []
  for j in input[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_tref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_tref.append(rs)
  with torch.no_grad():
    out = eval_inp(input[k:k+1],one_hot(labels[k:k+1],2).to(device),lengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_thyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_thyp.append(rs)
  k += 1
k = 0
while k < len(validation):
  l = []
  for j in validation[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_vref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_vref.append(rs)
  out = eval_inp(validation[k:k+1],one_hot(vlabels[k:k+1],2).to(device),vlengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_vhyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_vhyp.append(rs)
  k += 1
tscores = evaluator.get_scores(rouge_tref, rouge_thyp)
vscores = evaluator.get_scores(rouge_vref, rouge_vhyp)
print('BLEU Score Training')
print(nltk.translate.bleu_score.corpus_bleu(bleu_tref,bleu_thyp))
print('BLEU Score Validation')
print(nltk.translate.bleu_score.corpus_bleu(bleu_vref,bleu_vhyp))
print('ROUGE-1 Training')
print(tscores['rouge-1']['f'])
print('ROUGE-2 Training')
print(tscores['rouge-2']['f'])
print('ROUGE-1 Validation')
print(vscores['rouge-1']['f'])
print('ROUGE-2 Validation')
print(vscores['rouge-2']['f'])

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
device = torch.device('cuda:0')
#Cross aligned with LSTM and Attention,style as linear layer
class LSTM_ATN_CAE(nn.Module):
  def __init__(self, input_length, hidden_z,num_classes, len_style, attn_length, layers,vocab_length, gen_length,temp):
    super(LSTM_ATN_CAE,self).__init__()
    #number of classes
    self.numclasses = num_classes
    #generation length
    self.genlen = gen_length
    #hidden generator dim
    self.hidz = hidden_z
    #embedding dim
    self.inp = input_length
    #vocab size
    self.nvocab = vocab_length
    #encoder
    self.enc = nn.LSTM(input_length+len_style,hidden_z,layers,batch_first=True)
    #expanding style information
    self.style_layer = nn.Linear(num_classes,len_style)
    #generator
    self.dec = nn.LSTMCell(input_length+hidden_z+len_style,hidden_z)
    #attention linear layer - only need one because latent and encoder hidden dim are same length
    self.attn = nn.Linear(hidden_z,attn_length)
    #reduce to vector
    self.attn_reduce = nn.Linear(attn_length,1)
    #RELU activation
    self.relu = nn.ReLU()
    #number of layers
    self.numlayers = layers
    #word emebeddings
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #expand to vocab size
    self.llayer = nn.Linear(hidden_z,vocab_length)
    #softmax
    self.softmax = nn.Softmax(dim=1)
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #real labels
    oy = self.style_layer(one_hot(y,2).to(device))
    #transferred labels
    mody = self.style_layer(one_hot(1-y,2).to(device))
    #get word embeddings
    x = self.embedding(x)
    #pass through encoder
    ex = torch.cat((oy.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    out,(z,_) = self.enc(ex)
    out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
    z = z.view((z.size()[1],z.size()[2]*self.numlayers))
    #inputs for discriminator
    gen_fake = torch.zeros((x.size()[0],self.genlen,self.hidz)).to(device)
    gen_real = torch.zeros((x.size()[0],self.genlen,self.hidz)).to(device)
    #distribution over vocab for reconstruction loss
    gen = torch.zeros((x.size()[0],self.nvocab,self.genlen)).to(device)
    i = 0
    #iterate over generation length
    while i < self.genlen:
      if i == 0:
        #attention weights
        attn_weights = self.softmax(self.attn_reduce(self.relu(self.attn(out) + self.attn(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
        #apply weighting
        attn_vec = torch.bmm(attn_weights,out)
        attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
        #real and fake input to generator
        inp1 = torch.cat((attn_vec,oy,x[:,i,:]),1)
        inp2 = torch.cat((attn_vec,mody,x[:,i,:]),1)
      else:
        #need two attention mechanisms - one for teacher forced (real) and one for free running (fake)
        attn_weights_real = self.softmax(self.attn_reduce(self.relu(self.attn(out) + self.attn(z[0:cut].unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights_real = attn_weights_real.view((attn_weights_real.size()[0],1,attn_weights_real.size()[1]))
        attn_vec_real = torch.bmm(attn_weights_real,out)
        attn_vec_real = attn_vec_real.view((attn_vec_real.size()[0],attn_vec_real.size()[2]))
        attn_weights_fake = self.softmax(self.attn_reduce(self.relu(self.attn(out) + self.attn(z[cut:].unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights_fake = attn_weights_fake.view((attn_weights_fake.size()[0],1,attn_weights_fake.size()[1]))
        attn_vec_fake = torch.bmm(attn_weights_fake,out)
        attn_vec_fake = attn_vec_fake.view((attn_vec_fake.size()[0],attn_vec_fake.size()[2]))
        #real and fake input to generator
        inp1 = torch.cat((attn_vec_real,oy,x[:,i,:]),1)
        inp2 = torch.cat((attn_vec_fake,mody,all_embed),1)
      #pass through generator
      ip = torch.cat((inp1,inp2),0)
      z,_ = self.dec(ip)
      #get discriminator output
      cut = int(len(z)/2)
      gen_real[:,i,:] = z[0:cut]
      gen_fake[:,i,:] = z[cut:len(z)]
      #expand to vocab size
      kp = self.llayer(z)
      cut = int(len(kp)/2)
      #rescale
      kp = (kp - torch.max(kp,dim=1)[0].unsqueeze(1).repeat(1,kp.size()[1]))
      #store distribution
      gen[:,:,i] = kp[0:cut]
      #get weighted average words for fake
      su = torch.sum(torch.exp(kp[cut:len(kp)]/self.temp),axis=1)
      softmax = (torch.exp(kp[cut:len(kp)]/self.temp).T/(su)).T
      ta = torch.arange(self.nvocab).to(device)
      all_embed = self.embedding(ta)
      all_embed = softmax@all_embed
      i += 1
    return gen,gen_real,gen_fake
#discriminator
class Disc(nn.Module):
  def __init__(self, input_length,disc_hidden_length):
    super(Disc,self).__init__()
    self.disc = nn.LSTM(input_length,disc_hidden_length,batch_first=True,bidirectional=True)    
    self.disc_linear = nn.Linear(disc_hidden_length*2,2)
  def forward(self,disc_store):
    _,(hid,_) = self.disc(disc_store)
    hid = hid.reshape((hid.size()[1],hid.size()[2]*2))
    hid = self.disc_linear(hid)
    return hid

In [ ]:
#CAE with LSTM with attention, style as linear layer training
#set seeds for reproducibility
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
#choose which labels to use - two at a time
ind = np.array(author_labels)
a0 = np.where(ind==0)[0]
a1 = np.where(ind==2)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#sequence length
mlen = 15
#get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#divide into training and vaildaiton
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[3] = 0
weights[0] = 0
#initialize network, discriminators, loss and optimizers
net = LSTM_ATN_CAE(300,300,2,50,50,1,len(vocab.keys()),mlen-1,10)
disc_0 = Disc(300,100)
disc_1 = Disc(300,100)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
lf2 = nn.CrossEntropyLoss(reduction='mean')
net = net.to(device)
disc_0 = disc_0.to(device)
disc_1 = disc_1.to(device)
opt1 = torch.optim.Adam(net.parameters(),lr=0.0001)
opt2 = torch.optim.Adam(disc_0.parameters(),lr=0.0001)
opt3 = torch.optim.Adam(disc_1.parameters(),lr=0.0001)
optimizer = [opt3,opt2,opt1]
models = [disc_0,disc_1,net]
trainloss = []
traindisc = []
valloss = []
td = []
batch_size=256
i = 0
while i < 500:
  #sample random minibatch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #inputs for network
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    oh = inp[:,1:mlen].long().to(device)
    curlab = labels[train_track[k*batch_size:(k+1)*batch_size]]
    #pass through network
    gen,gen_real,gen_fake = net(inp,curlab,les)
    #find where labels are 0 and 1
    zero_labels = torch.where(curlab==0)[0]
    one_labels = torch.where(curlab==1)[0]
    #three possibilities - all 0, all 1, both
    if len(zero_labels)==0 and len(one_labels) > 0:
      #discriminator output
      out0 = disc_0(gen_real)
      out1 = disc_1(gen_fake)
      #discriminator labels
      lab0 = torch.ones(len(gen_real)).long().to(device)
      lab1 = torch.zeros(len(gen_fake)).long().to(device)
      #output to check if discriminator is fooled
      zero_g = None
      one_g = disc_1(gen_fake)
      #fake labels
      lab0_g = None
      lab1_g = torch.ones(len(gen_fake)).long().to(device)
    elif len(one_labels)==0 and len(zero_labels) > 0:
      #discriminator output
      out0 = disc_0(gen_fake)
      out1 = disc_1(gen_real)
      #discriminator labels
      lab0 = torch.ones(len(gen_fake)).long().to(device)
      lab1 = torch.zeros(len(gen_real)).long().to(device)
      #output to check if discriminator is fooled
      zero_g = disc_0(gen_fake)
      one_g = None
      #fake labels
      lab1_g = None
      lab0_g = torch.ones(len(gen_fake)).long().to(device)
    else:
      #put all 0 and all 1 samples together
      g0 = torch.cat((gen_real[zero_labels],gen_fake[one_labels]),0)
      g1 = torch.cat((gen_real[one_labels],gen_fake[zero_labels]),0)
      #discriminator output
      out0 = disc_0(g0)
      out1 = disc_1(g1)
      #discriminator labels
      lab0 = torch.cat((torch.ones(len(zero_labels)),torch.zeros(len(one_labels))),0).long().to(device)
      lab1 = torch.cat((torch.ones(len(one_labels)),torch.zeros(len(zero_labels))),0).long().to(device)
      #output to check if discriminator is fooled
      zero_g = disc_0(gen_fake[one_labels])
      one_g = disc_1(gen_fake[zero_labels])
      #fake labels
      lab0_g = torch.ones(len(one_labels)).long().to(device)
      lab1_g = torch.ones(len(zero_labels)).long().to(device)
    #discriminator loss
    l0 = lf2(out0,lab0)
    l1 = lf2(out1,lab1)
    #loss for fooling discriminator
    if zero_g is not None:
      l0_g = lf2(zero_g,lab0_g)
    else:
      l0_g = 0
    if one_g is not None:
      l1_g = lf2(one_g,lab1_g)
    else:
      l1_g = 0
    #reconstruction loss
    rec_loss = loss_func(gen,oh)
    loss = torch.mean(rec_loss) + (l0_g + l1_g)
    losses = [l0,l1,loss]
    #backward gradient
    g = 0
    while g < 3:
      optimizer[g].zero_grad()
      losses[g].backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm_(models[g].parameters(),max_norm=1)
      g += 1
    g = 0
    while g < 3:
      optimizer[g].step()
      g += 1
    #record loss
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    k += 1
  trainloss.append(lp/len(input))
  #check validation error
  with torch.no_grad():
    k = 0
    lp = 0
    while k < len(validation)//256:
      inp = validation[k*256:(k+1)*256]
      oh = inp[:,1:mlen].long()
      class_inp = one_hot(vlabels[k*256:(k+1)*256],2).to(device)
      le = vlengs[k*256:(k+1)*256]
      y = net.style_layer(class_inp)
      x = net.embedding(inp)
      ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
      ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
      out,(z,_) = net.enc(ex)
      out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
      z = z.view((z.size()[1],z.size()[2]*net.numlayers))
      gen = torch.zeros((x.size()[0],net.nvocab,net.genlen)).to(device)
      p = 0
      while p < net.genlen:
        attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn(out) + net.attn(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
        attn_vec = torch.bmm(attn_weights,out)
        attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
        ip = torch.cat((attn_vec,y,x[:,p,:]),1)
        z,_ = net.dec(ip)
        kp = net.llayer(z)
        gen[:,:,p] = kp
        p += 1
      rec_loss = loss_func(gen,oh)
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      k += 1
  valloss.append(lp/len(validation))  
  print(i,trainloss[-1],valloss[-1])
  i += 1

In [ ]:
#calculate BLEU and ROUGE for CAE with attention
import nltk
import rouge
nltk.download('punkt')
apply_avg = True
apply_best = False
evaluator = rouge.Rouge(metrics=['rouge-n'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
def eval_inp(inp,class_inp,le):
  with torch.no_grad():
    y = net.style_layer(class_inp)
    x = net.embedding(inp)
    ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
    out,(hz,_) = net.enc(ex)
    out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
    z = hz.view((hz.size()[1],hz.size()[2]*net.numlayers))
    gen = torch.zeros((hz.size()[0],net.nvocab,net.genlen)).to(device)
    p = 0
    while p < net.genlen:
      attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn(out) + net.attn(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
      attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
      attn_vec = torch.bmm(attn_weights,out)
      attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
      ip = torch.cat((attn_vec,y,x[:,p,:]),1)
      z,_ = net.dec(ip)
      hid = net.llayer(z)
      gen[:,:,p] = hid
      p += 1
  return gen
bleu_tref = []
bleu_thyp = []
bleu_vref = []
bleu_vhyp = []
rouge_tref = []
rouge_thyp = []
rouge_vref = []
rouge_vhyp = []
ke = list(vocab.keys())
k = 0
while k < len(input):
  l = []
  for j in input[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_tref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_tref.append(rs)
  out = eval_inp(input[k:k+1],one_hot(labels[k:k+1],2).to(device),lengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_thyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_thyp.append(rs)
  k += 1
k = 0
while k < len(validation):
  l = []
  for j in validation[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_vref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_vref.append(rs)
  out = eval_inp(validation[k:k+1],one_hot(vlabels[k:k+1],2).to(device),vlengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_vhyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_vhyp.append(rs)
  k += 1
tscores = evaluator.get_scores(rouge_tref, rouge_thyp)
vscores = evaluator.get_scores(rouge_vref, rouge_vhyp)
print('BLEU Score Training')
print(nltk.translate.bleu_score.corpus_bleu(bleu_tref,bleu_thyp))
print('BLEU Score Validation')
print(nltk.translate.bleu_score.corpus_bleu(bleu_vref,bleu_vhyp))
print('ROUGE-1 Training')
print(tscores['rouge-1']['f'])
print('ROUGE-2 Training')
print(tscores['rouge-2']['f'])
print('ROUGE-1 Validation')
print(vscores['rouge-1']['f'])
print('ROUGE-2 Validation')
print(vscores['rouge-2']['f'])

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn.functional import softmax
device = torch.device('cuda:0')
#Cross aligned with bidirectional LSTM and attention,style as linear layer
class LSTM_ATN_CAE_BD(nn.Module):
  def __init__(self, input_length, hidden_z,num_classes, len_style, attn_length, layers,vocab_length, gen_length,temp):
    super(LSTM_ATN_CAE_BD,self).__init__()
    #number of classes
    self.numclasses = num_classes
    #generation length
    self.genlen = gen_length
    #embedding dim
    self.inp = input_length
    #hidden generator dim
    self.hidz = 2*hidden_z
    #vocab length
    self.nvocab = vocab_length
    #encoder
    self.enc = nn.LSTM(input_length+len_style,hidden_z,layers,batch_first=True,bidirectional=True)
    #style information layer
    self.style_layer = nn.Linear(num_classes,len_style)
    #decoder
    self.dec = nn.LSTMCell(input_length+2*hidden_z+len_style,2*hidden_z)
    #attention layer - only one since hidden states same dim
    self.attn = nn.Linear(2*hidden_z,attn_length)
    #reduce attention to vector
    self.attn_reduce = nn.Linear(attn_length,1)
    #ReLU activation
    self.relu = nn.ReLU()
    #word embeddings
    self.embedding = nn.Embedding(vocab_length,input_length, padding_idx=0)
    #expand to vocab length
    self.llayer = nn.Linear(2*hidden_z,vocab_length)
    #number of layers
    self.numlayers = layers
    #softmax
    self.softmax = nn.Softmax(dim=1)
    #temperature
    self.temp = temp
  def forward(self,x,y,les):
    #real labels
    oy = self.style_layer(one_hot(y,2).to(device))
    #transferred labels
    mody = self.style_layer(one_hot(1-y,2).to(device))
    #get word embeddings
    x = self.embedding(x)
    #pass through encoder
    ex = torch.cat((oy.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,les,batch_first=True,enforce_sorted=False)
    out,(z,_) = self.enc(ex)
    out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
    z = z.view((z.size()[1],z.size()[2]*2*self.numlayers))
    #transferred samples
    gen_fake = torch.zeros((x.size()[0],self.genlen,self.hidz)).to(device)
    #real samples
    gen_real = torch.zeros((x.size()[0],self.genlen,self.hidz)).to(device)
    #distribution over vocab
    gen = torch.zeros((x.size()[0],self.nvocab,self.genlen)).to(device)
    i = 0
    while i < self.genlen:
      if i == 0:
        #at first step we have same initial - create attention weights
        attn_weights = self.softmax(self.attn_reduce(self.relu(self.attn(out) + self.attn(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
        #apply weighting
        attn_vec = torch.bmm(attn_weights,out)
        attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
        #create real and transferred input
        inp1 = torch.cat((attn_vec,oy,x[:,i,:]),1)
        inp2 = torch.cat((attn_vec,mody,x[:,0,:]),1)
      else:
        #attention for real samples
        attn_weights_real = self.softmax(self.attn_reduce(self.relu(self.attn(out) + self.attn(z[0:cut].unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights_real = attn_weights_real.view((attn_weights_real.size()[0],1,attn_weights_real.size()[1]))
        attn_vec_real = torch.bmm(attn_weights_real,out)
        attn_vec_real = attn_vec_real.view((attn_vec_real.size()[0],attn_vec_real.size()[2]))
        #attention for transferred samples
        attn_weights_fake = self.softmax(self.attn_reduce(self.relu(self.attn(out) + self.attn(z[cut:].unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights_fake = attn_weights_fake.view((attn_weights_fake.size()[0],1,attn_weights_fake.size()[1]))
        attn_vec_fake = torch.bmm(attn_weights_fake,out)
        attn_vec_fake = attn_vec_fake.view((attn_vec_fake.size()[0],attn_vec_fake.size()[2]))
        #create real and transferred output
        inp1 = torch.cat((attn_vec_real,oy,x[:,i,:]),1)
        inp2 = torch.cat((attn_vec_fake,mody,all_embed),1)
      #pass through decoder
      ip = torch.cat((inp1,inp2),0)
      z,_ = self.dec(ip)
      #get discriminator output
      cut = int(len(z)/2)
      gen_real[:,i,:] = z[0:cut]
      gen_fake[:,i,:] = z[cut:len(z)]
      #expand to vocab size
      kp = self.llayer(z)
      #rescale
      kp = (kp - torch.max(kp,dim=1)[0].unsqueeze(1).repeat(1,kp.size()[1]))
      #store distribution
      gen[:,:,i] = kp[0:cut]
      #get weighted average words for fake
      su = torch.sum(torch.exp(kp[cut:len(kp)]/self.temp),axis=1)
      softmax = (torch.exp(kp[cut:len(kp)]/self.temp).T/(su)).T
      ta = torch.arange(self.nvocab).to(device)
      all_embed = self.embedding(ta)
      all_embed = softmax@all_embed
      i += 1
    return gen,gen_real,gen_fake
#discriminator
class Disc(nn.Module):
  def __init__(self, input_length,disc_hidden_length):
    super(Disc,self).__init__()
    self.disc = nn.LSTM(input_length,disc_hidden_length,batch_first=True,bidirectional=True)    
    self.disc_linear = nn.Linear(disc_hidden_length*2,2)
  def forward(self,disc_store):
    _,(hid,_) = self.disc(disc_store)
    hid = hid.reshape((hid.size()[1],hid.size()[2]*2))
    hid = self.disc_linear(hid)
    return hid
    return gen_store,disc_store


In [ ]:
#CAE with BD LSTM and attention training
#set seeds for deterministic output
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#choose labels we want to use - two at a time
a0 = np.where(ind==2)[0]
a1 = np.where(ind==3)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
#Sequence length
mlen = 15
#Get vocab, labels, lengths, and numbered input for network
vocab = create_vocab_dict(avail_poems,mincount=0)
padded,lab,lengs = vnorm(avail_poems,vocab,avail_ind,mlen)
#divide into training and validation
fp = np.array(lab)
_,rc = np.unique(fp,return_counts=True)
cut = np.array(np.round(rc*0.8),dtype=int)
w0 = np.where(fp==0)[0]
w1 = np.where(fp==1)[0]
np.random.shuffle(w0)
np.random.shuffle(w1)
trind =  np.concatenate((w0[0:cut[0]],w1[0:cut[1]]))
vind = np.concatenate((w0[cut[0]:len(w0)],w1[cut[1]:len(w1)]))
train_track = np.arange(len(trind))
valid_track = np.arange(len(vind))
lof = np.array(lengs)
lengs = lof[trind]
vlengs = lof[vind]
input = torch.tensor(padded)[trind].to(device)
validation = torch.tensor(padded)[vind].to(device)
labels = torch.tensor(lab)[trind].to(device)
vlabels = torch.tensor(lab)[vind].to(device)
#Weighting to ignore gradient over padding and unk tokens
weights = torch.ones(len(vocab.keys())).to(device)
weights[3] = 0
weights[0] = 0
#initialize network,discriminators, loss, and optimizers
net = LSTM_ATN_CAE_BD(300,300,2,50,50,1,len(vocab.keys()),mlen-1,10)
disc_0 = Disc(600,100)
disc_1 = Disc(600,100)
loss_func = nn.CrossEntropyLoss(weight=weights,reduction='none')
lf2 = nn.CrossEntropyLoss(reduction='mean')
net = net.to(device)
disc_0 = disc_0.to(device)
disc_1 = disc_1.to(device)
coef = np.concatenate((np.zeros(40),np.linspace(0,1,30),np.ones(940)))
opt1 = torch.optim.Adam(net.parameters(),lr=0.0001)
opt2 = torch.optim.Adam(disc_0.parameters(),lr=0.0001)
opt3 = torch.optim.Adam(disc_1.parameters(),lr=0.0001)
optimizer = [opt3,opt2,opt1]
models = [disc_0,disc_1,net]
trainloss = []
traindisc = []
valloss = []
td = []
batch_size=256
i = 0
while i < 76:
  #sample random minibatch
  np.random.shuffle(train_track)
  k = 0
  lp = 0
  klp = 0
  while k < len(trind)//batch_size + 1:
    #get inputs
    inp = input[train_track[k*batch_size:(k+1)*batch_size]]
    les = lengs[train_track[k*batch_size:(k+1)*batch_size]]
    oh = inp[:,1:mlen].long().to(device)
    curlab = labels[train_track[k*batch_size:(k+1)*batch_size]]
    #pass through network
    gen,gen_real,gen_fake = net(inp,curlab,les)
    #get 0 and 1 labels indices
    zero_labels = torch.where(curlab==0)[0]
    one_labels = torch.where(curlab==1)[0]
    if len(zero_labels)==0 and len(one_labels) > 0:
      #discirminator output
      out0 = disc_0(gen_real)
      out1 = disc_1(gen_fake)
      #discriminator output
      lab0 = torch.ones(len(gen_real)).long().to(device)
      lab1 = torch.zeros(len(gen_fake)).long().to(device)
      #output to see if fake fools discriminator
      zero_g = None
      one_g = disc_1(gen_fake)
      #fake labels
      lab0_g = None
      lab1_g = torch.ones(len(gen_fake)).long().to(device)
    elif len(one_labels)==0 and len(zero_labels) > 0:
      #discriminator output
      out0 = disc_0(gen_fake)
      out1 = disc_1(gen_real)
      #discriminator labels
      lab0 = torch.ones(len(gen_fake)).long().to(device)
      lab1 = torch.zeros(len(gen_real)).long().to(device)
      #output to see if fake fools discriminator
      zero_g = disc_0(gen_fake)
      one_g = None
      #fake labels
      lab1_g = None
      lab0_g = torch.ones(len(gen_fake)).long().to(device)
    else:
      #put 0 and 1 together
      g0 = torch.cat((gen_real[zero_labels],gen_fake[one_labels]),0)
      g1 = torch.cat((gen_real[one_labels],gen_fake[zero_labels]),0)
      #discriminator output
      out0 = disc_0(g0)
      out1 = disc_1(g1)
      #discriminator labels
      lab0 = torch.cat((torch.ones(len(zero_labels)),torch.zeros(len(one_labels))),0).long().to(device)
      lab1 = torch.cat((torch.ones(len(one_labels)),torch.zeros(len(zero_labels))),0).long().to(device)
      #output to see if fake fools discriminator
      zero_g = disc_0(gen_fake[one_labels])
      one_g = disc_1(gen_fake[zero_labels])
      #fake labels
      lab0_g = torch.ones(len(one_labels)).long().to(device)
      lab1_g = torch.ones(len(zero_labels)).long().to(device)
    #discriminator loss
    l0 = lf2(out0,lab0)
    l1 = lf2(out1,lab1)
    #loss on fooling discriminator
    if zero_g is not None:
      l0_g = lf2(zero_g,lab0_g)
    else:
      l0_g = 0
    if one_g is not None:
      l1_g = lf2(one_g,lab1_g)
    else:
      l1_g = 0
    #reconstruction loss
    rec_loss = loss_func(gen,oh)
    loss = torch.mean(rec_loss) + (l0_g + l1_g)
    losses = [l0,l1,loss]
    #backward gradients
    g = 0
    while g < 3:
      optimizer[g].zero_grad()
      losses[g].backward(retain_graph=True)
      torch.nn.utils.clip_grad_norm_(models[g].parameters(),max_norm=1)
      g += 1
    g = 0
    while g < 3:
      optimizer[g].step()
      g += 1
    #record loss
    mask = rec_loss != 0
    lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
    k += 1
  trainloss.append(lp/len(input))
  #check validation error
  with torch.no_grad():
    k = 0
    lp = 0
    while k < len(validation)//256:
      inp = validation[k*256:(k+1)*256]
      oh = inp[:,1:mlen].long()
      class_inp = one_hot(vlabels[k*256:(k+1)*256],2).to(device)
      le = vlengs[k*256:(k+1)*256]
      y = net.style_layer(class_inp)
      x = net.embedding(inp)
      ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
      ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
      out,(z,_) = net.enc(ex)
      out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
      z = z.view((z.size()[1],z.size()[2]*2*net.numlayers))
      gen = torch.zeros((x.size()[0],net.nvocab,net.genlen)).to(device)
      p = 0
      while p < net.genlen:
        attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn(out) + net.attn(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
        attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
        attn_vec = torch.bmm(attn_weights,out)
        attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
        ip = torch.cat((attn_vec,y,x[:,p,:]),1)
        z,_ = net.dec(ip)
        kp = net.llayer(z)
        z = z[0:1]
        gen[:,:,p] = kp
        p += 1
      rec_loss = loss_func(gen,oh)
      mask = rec_loss != 0
      lp += torch.sum(torch.sum(mask*rec_loss,1)/torch.sum(mask,1)).item()
      k += 1
  valloss.append(lp/len(validation))  
  print(i,trainloss[-1],valloss[-1])
  i += 1

In [ ]:
#get BLEU and ROUGE
import nltk
import rouge
nltk.download('punkt')
apply_avg = True
apply_best = False
evaluator = rouge.Rouge(metrics=['rouge-n'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
def eval_inp(inp,class_inp,le):
  with torch.no_grad():
    y = net.style_layer(class_inp)
    x = net.embedding(inp)
    ex = torch.cat((y.unsqueeze(1).repeat(1,x.size()[1],1),x),2)
    ex = pack_padded_sequence(ex,le,batch_first=True,enforce_sorted=False)
    out,(z,_) = net.enc(ex)
    out = pad_packed_sequence(out,batch_first=True,total_length=x.size()[1])[0]
    z = z.view((z.size()[1],z.size()[2]*2*net.numlayers))
    gen = torch.zeros((x.size()[0],net.nvocab,net.genlen)).to(device)
    p = 0
    while p < net.genlen:
      attn_weights = net.softmax(net.attn_reduce(net.relu(net.attn(out) + net.attn(z.unsqueeze(1).repeat(1,out.size()[1],1)))))
      attn_weights = attn_weights.view((attn_weights.size()[0],1,attn_weights.size()[1]))
      attn_vec = torch.bmm(attn_weights,out)
      attn_vec = attn_vec.view((attn_vec.size()[0],attn_vec.size()[2]))
      ip = torch.cat((attn_vec,y,x[:,p,:]),1)
      z,_ = net.dec(ip)
      kp = net.llayer(z)
      gen[:,:,p] = kp
      p += 1
  return gen
bleu_tref = []
bleu_thyp = []
bleu_vref = []
bleu_vhyp = []
rouge_tref = []
rouge_thyp = []
rouge_vref = []
rouge_vhyp = []
ke = list(vocab.keys())
k = 0
while k < len(input):
  l = []
  for j in input[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_tref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_tref.append(rs)
  out = eval_inp(input[k:k+1],one_hot(labels[k:k+1],2).to(device),lengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_thyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_thyp.append(rs)
  k += 1
k = 0
while k < len(validation):
  l = []
  for j in validation[k]:
    if j.cpu().item() == 2:
      pass
    elif j.cpu().item() == 1:
      break
    else:
      l.append(ke[j.cpu().item()])
  bleu_vref.append([l])
  i = 0
  rs = ""
  while i < len(l):
    if i == len(l) - 1:
      rs += l[i]
    else:
      rs += l[i] + ' '
    i += 1
  rouge_vref.append(rs)
  out = eval_inp(validation[k:k+1],one_hot(vlabels[k:k+1],2).to(device),vlengs[k:k+1])
  out = out.reshape((out.size()[1],-1))
  sents = torch.max(out,0)[1]
  gsen = []
  p = 0
  while p < len(sents):
    if sents[p].cpu().item() == 1:
      break
    else:
      gsen.append(ke[sents[p].cpu().item()])
    p += 1
  bleu_vhyp.append(gsen)
  i = 0
  rs = ""
  while i < len(gsen):
    if i == len(gsen) - 1:
      rs += gsen[i]
    else:
      rs += gsen[i] + ' '
    i += 1
  rouge_vhyp.append(rs)
  k += 1
tscores = evaluator.get_scores(rouge_tref, rouge_thyp)
vscores = evaluator.get_scores(rouge_vref, rouge_vhyp)
print('BLEU Score Training')
print(nltk.translate.bleu_score.corpus_bleu(bleu_tref,bleu_thyp))
print('BLEU Score Validation')
print(nltk.translate.bleu_score.corpus_bleu(bleu_vref,bleu_vhyp))
print('ROUGE-1 Training')
print(tscores['rouge-1']['f'])
print('ROUGE-2 Training')
print(tscores['rouge-2']['f'])
print('ROUGE-1 Validation')
print(vscores['rouge-1']['f'])
print('ROUGE-2 Validation')
print(vscores['rouge-2']['f'])

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

In [ ]:
!pip install transformers

In [ ]:
#gpt2
import nltk
import rouge
nltk.download('punkt')
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
torch.backends.cudnn.deterministic = True
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed(100)
import os
apply_avg = True
apply_best = False
evaluator = rouge.Rouge(metrics=['rouge-n'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
CUDA_LAUNCH_BLOCKING=1
ind = np.array(author_labels)
#choose labels we want to use - two at a time
a0 = np.where(ind==0)[0]
a1 = np.where(ind==1)[0]
ind[a0] = 0
ind[a1] = 1
avail_poems = np.array(r_authors_poem_line,dtype='object')[np.concatenate((a0,a1))]
avail_ind = ind[np.concatenate((a0,a1))]
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
bleu_ref = []
bleu_hyp = []
rouge_ref = []
rouge_hyp = []
ce = torch.nn.CrossEntropyLoss()
lp = 0
i = 0
ct = 0
while i < len(avail_poems):
  for line in avail_poems[i]:
    if len(line) > 0:
      ct += 1
      bleu_ref.append(line)
      st=""
      j = 0
      while j < len(line):
        if j == len(line) - 1:
          st += line[j]
        else:
          st += line[j] + ' '
        j += 1
      rouge_ref.append(st)
      with torch.no_grad():
        inputs = tokenizer(line,is_split_into_words=True,return_tensors='pt',max_length=15,truncation=True).to(device)
        truth = inputs['input_ids']
        outputs = model(**inputs,labels=inputs['input_ids'])
        logits = outputs.logits
      logits = logits.view((logits.size()[0],logits.size()[2],logits.size()[1]))
      maxlog = torch.max(logits,1)[1].cpu().reshape(-1).tolist()
      gst = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(maxlog))
      rouge_hyp.append(gst)
      bleu_hyp.append(gst.split())
      lp += ce(logits,truth)
  print(i)
  i += 1
print(lp/ct)
tscores = evaluator.get_scores(rouge_ref, rouge_hyp)
print('BLEU Score')
print(nltk.translate.bleu_score.corpus_bleu(bleu_ref,bleu_hyp))
print('ROUGE-1')
print(tscores['rouge-1']['f'])
print('ROUGE-2')
print(tscores['rouge-2']['f'])

In [ ]:
!kill -9 -1